In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 64.0 MB/s eta 0:00:00


In [10]:
!pwd

/home/ec2-user/SageMaker


In [3]:
import os
from pathlib import Path

from ultralytics.utils.downloads import download

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/ec2-user/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [11]:
import os

# Create dataset directory in SageMaker environment
os.makedirs('/home/ec2-user/SageMaker/datasets/VisDrone', exist_ok=True)

In [ ]:
from pathlib import Path
import os
from pathlib import Path
from ultralytics.utils.downloads import download
from PIL import Image
from tqdm import tqdm

dataset_root = Path('/home/ec2-user/SageMaker/datasets/VisDrone')


# URLs for the dataset
urls = [
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-train.zip',
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-val.zip',
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-dev.zip',
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-challenge.zip'
]

# Download the dataset
download(urls, dir=dataset_root, curl=True, threads=4)

# Convert annotations
def visdrone2yolo(dir):
    def convert_box(size, box):
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    (dir / 'labels').mkdir(parents=True, exist_ok=True)  # make labels directory
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # Ignore 'ignored regions' class
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                with open(str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}'), 'w') as fl:
                    fl.writelines(lines)

# Convert annotations for train, val, and test-dev sets
for d in ['VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev']:
    visdrone2yolo(dataset_root / d)  # Convert VisDrone annotations to YOLO labels

Converting /home/ec2-user/SageMaker/datasets/VisDrone/VisDrone2019-DET-test-dev: 1610it [01:02, 25.73it/s]


In [ ]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO("yolo11l.pt")

# Train the model using multiple GPUs
results = model.train(
    data="VisDrone.yaml",  # Path to dataset configuration
    epochs=100,
    batch=18,
    imgsz=640,
    device="0,1,2,3,4,5"  # Specify multiple GPU devices (e.g., GPU 0, 1, and 2)
)


Ultralytics 8.3.38 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:1 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:2 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:3 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:4 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:5 (NVIDIA A100-SXM4-40GB, 40326MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=VisDrone.yaml, epochs=100, time=None, patience=100, batch=18, imgsz=640, save=True, save_period=-1, cache=False, device=0,1,2,3,4,5, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_sca

100%|██████████| 755k/755k [00:00<00:00, 77.4MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 261MB/s]


AMP: checks passed ✅


...

                   all        548      38759      0.579      0.445      0.465      0.287

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      3.59G     0.9983     0.5454     0.8416         92        640: 100%|██████████| 360/360 [00:59<00:00,  6.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 92/92 [00:05<00:00, 16.50it/s]


                   all        548      38759      0.576      0.448      0.465      0.287

100 epochs completed in 1.834 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 51.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 51.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.38 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:1 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:2 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:3 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:4 (NVIDIA A100-SXM4-40GB, 40326MiB)
                                                 CUDA:5 (NVIDIA A100-SXM4-40GB, 40326MiB)
YOLO11l summary (fused): 464 layers, 25,287,022 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 92/92 [00:08<00:00, 10.57it/s]


                   all        548      38759      0.585      0.453      0.471      0.292
            pedestrian        520       8844       0.64      0.457      0.522      0.253
                people        482       5125      0.649      0.327      0.398      0.164
               bicycle        364       1287      0.384      0.252      0.228      0.105
                   car        515      14064        0.8      0.799      0.832      0.613
                   van        421       1975      0.572      0.497      0.505      0.364
                 truck        266        750      0.618      0.461      0.469      0.319
              tricycle        337       1045      0.481      0.381      0.365      0.212
       awning-tricycle        220        532      0.384      0.216      0.206      0.133
                   bus        131        251       0.75      0.618      0.649        0.5
                 motor        485       4886      0.572      0.519      0.536      0.257
Speed: 0.1ms preproce